# Summer School: Digitale Methoden der Zeitungsanalyse

Dieses Jupyter Notebook zeigt, wie Zeitungstitel (am Beispiel der Zeitung „[Die Radlerin](https://www.deutsche-digitale-bibliothek.de/newspaper/278213)) aus dem Zeitungsportal über die Programmierschnittstelle (API) der Deutschen Digitale Bibliothek heruntergeladen werden können. Dazu werden Aspekte der API erklärt und der Zugriff mit Python-Programmcode demonstriert.

## Suchindizes der Deutsche Digitale Bibliothek
Die Deutsche Digitale Bibliothek betreibt verschiedene [Solr](https://solr.apache.org/guide/8_8/searching.html)-Suchindizes, die für die verschiedenen Funktionen der (Sub-) Portale benötigt werden. Das Zeitungsportal benutzt zwei Suchindizes. Eine weiterführende Dokumentation befindet sich hier: https://api.deutsche-digitale-bibliothek.de/#/search/getSolrSearch

- `newspaper`: enthält Informationen über Zeitungstitel
  - Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/schema.xml
  - Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/solrconfig.xml
- `newspaper-issues`: enthält die zeitungsbezogenen Metadaten inkl. Volltexte
  - Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/schema.xml
  - Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/solrconfig.xml

## Suchindex `newspaper`
Der Suchindex `newspaper` ist ein Suchindex über alle Zeitungstitel der [Zeitschriftendatenbank (ZDB)](https://zdb-katalog.de/). Eine `*`-Suche kann folgendermaßen durchgeführt werden:

- [https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=\*:\*](https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=*:*)

Im Schema des Suchindex (s.o.) sind die Suchfelder (Facetten) dokumentiert. Wenn nur Zeitungstitel gefunden werden sollen, die im Zeitungsportal verfügbar sind, dann muss nach `hasLoadedIssues:true` (feld:wert) gesucht werden.

- https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true

Die Suchfelder können beliebig miteinander kombiniert werden. Das geht mit den Operatoren `AND` und `OR`. Wenn man Zeitungstitel mit „Radlerin“ suchen möchte, die auch im Zeitungsportal verfügbar sind, dann kombiniert man: `hasLoadedIssues:true AND title:"radlerin"`

- [https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true AND title:"radlerin"](https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true%20AND%20title:"radlerin")

### Python-Programmcode

Python-Programmcode in Jupyter Notebooks ermöglicht interaktives Programmieren und sofortige Anzeige von Ergebnissen. Es ist ideal für die Datenanalyse und Visualisierungen. Python-Bibliotheken können in Notebooks nachgenutzt werden und erhöhen so den Funktionsumfang enorm.

Die o.g Solr-Abfragen können mit Python ausgeführt werden ([`requests`](https://pypi.org/project/requests/)) und die Antwort von der API mit einem JSON-Parser (`json`) gelesen werden. Eine andere Möglichkeit ist, den Solr-Client [`pysolr`](https://pypi.org/project/pysolr/) zu benutzen. Dieser muss zunächst in der Umgebung mit `pip install -q pysolr` (oder ggf. mit Conda: `conda install conda-forge::pysolr`) installiert werden.

In [27]:
# Python-Bibliothek pysolr installieren
%pip install -q pysolr

Note: you may need to restart the kernel to use updated packages.


### KI-generierter Programmcode

Die Erstellung des Python-Programmcodes ist KI-gestützt möglich. Die folgenden KI-Prompts sind mit ChatGPT (GPT-4o) erfolgreich getestet und liefern (oft 😉) das gewünschte Ergebnis.

<div class="alert alert-block alert-info">
<b>Prompt:</b> Ich möchte in Python mit dem Solr-Client <code>pysolr</code> auf den Endpunkt https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper zugreifen. Kannst Du mir einen Python-Code erstellen, der im Feld <code>title</code> nach „Radlerin“ sucht und auch <code>hasLoadedIssues</code> auf wahr setzt. Gibt bitte <code>id</code>, <code>title</code>, <code>location</code>, <code>frequency</code> und <code>progress</code> für jeden Suchtreffer aus.
</div>

In [28]:
import pysolr

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'title:Radlerin AND hasLoadedIssues:true',
    'fl': 'id,title,location,frequency,progress',
    'rows': 10  # Anzahl der zurückzugebenden Ergebnisse
}

# Suche ausführen
results = solr.search(**q)

# Ergebnisse ausgeben
for result in results:
    print(f"ID: {result.get('id', 'N/A')}")
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Location: {result.get('location', 'N/A')}")
    print(f"Frequency: {result.get('frequency', 'N/A')}")
    print(f"Progress: {result.get('progress', 'N/A')}")
    print("-" * 40)

ID: 2782137-7
Title: ['Die Radlerin']
Location: N/A
Frequency: ['http://id.loc.gov/vocabulary/frequencies/smn']
Progress: ['1. Jahrgang, No. 12 (25. Februar 1897)-1. Jahrgang, No. 14 (25. März 1897) ; 1. Jahrgang, No. 21 (10. Juli 1897)-1. Jahrgang, No. 23 (10. August 1897) ;  2. Jahrgang, No. 10 (25. Februar 1898) ; 2. Jahrgang, No. 12 (25. März 1898)-2. Jahrgang, No. 14 (25. April 1898) ; 3. Jahrgang, No. 1 (15. Oktober 1898)-3. Jahrgang, No. 6 (30. Dezember 1898)']
----------------------------------------
ID: 2782140-7
Title: ['Radlerin u. Radler']
Location: N/A
Frequency: ['http://id.loc.gov/vocabulary/frequencies/smn']
Progress: ['3. Jahrgang, No. 7 (15. Januar 1899)-3. Jahrgang, No. 24 (30. September 1899) ; 4. Jahrgang, No. 2 (30. Oktober 1899)-4.Jahrgang, No. 3 (15. November 1899) ; 4. Jahrgang, No. 23 (15. September 1900) ; 5. Jahrgang, No. 1 (15. Oktober 1900)-7. Jahrgang, No. 1 (15. Januar 1903) ; 7. Jahrgang, No. 4 (15. März 1903)-7. Jahrgang, No. 6 (31. März 1903)']
------

## Suchindex `newspaper-issues`

Der Suchindex `newspaper-issues` ist ein weiterer Suchindex des Zeitungsportals. Dieser enthält alle Ausgaben (`type:issue`) einer Zeitung und alle Seiten (`type:page`). Wenn man nur in einem Zeitungstitel suchen möchte, so kann dies über `zdb_id:<ID aus der Zeitschriftendatenbank>` (für die „Radlerin“ ist es `zdb_id:2782137-7`) eingrenzen.

### KI-generierter Programmcode

<div class="alert alert-block alert-info">
<b>Prompt:</b> Schreibe ein Python-Code, das mithilfe der <code>pysolr</code>-Bibliothek eine Suche in einem Solr-Index durchführt und die Ergebnisse in ein Pandas DataFrame überführt. Der Solr-Index ist über die URL https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues erreichbar. Die Suchabfrage soll nach Dokumenten mit der <code>zdb_id</code> „2782137-7“ und dem <code>type</code> „issue“ suchen und bis zu 1000 Ergebnisse zurückgeben. Anschließend sollen die Ergebnisse in ein Pandas DataFrame überführt und angezeigt werden.
</div>

In [29]:
import pysolr
import pandas as pd

solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'
solr = pysolr.Solr(solr_url, always_commit=True, timeout=10)

q = {
    'q': 'zdb_id:2782137-7 AND type:issue',
    'rows': 1000
}

response = solr.search(**q)
    
# Überführen der Ergebnisse in ein Pandas DataFrame
df = pd.DataFrame(response.docs)

# DataFrame anzeigen
df

,id,paper_title,provider_ddb_id,provider,zdb_id,ns_disclaimer_required,publication_date,language,thumbnail
0,HG7ELL5U3S2JP3NB5YN7HDH6TLUN2FJ7,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1898-12-30T12:00:00Z,[ger],bc4fb2f5-cb92-413b-82bc-8314798647c8
1,6N7GKKGH5IPAYYTBNHCG6RM4PZ6F6TWF,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1897-02-25T12:00:00Z,[ger],9b49e58b-86cc-49a8-a2d2-864903747343
2,F47A6H5QGYWDZE5JVHTUPCVI5TJWDBFX,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1897-03-25T12:00:00Z,[ger],64a6c1c2-af95-4d2e-9da0-7d20be6b5244
3,KUVZNUNQH5S4P44J76HYAKBQUAFV5LSI,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1897-07-10T12:00:00Z,[ger],9c5b70f9-f91b-4167-af25-7266ec4edf94
4,TPV722VIMVEUEHQJJARAX7PFZOVNYRMQ,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1898-04-10T12:00:00Z,[ger],305a513c-1c2a-414f-b65d-05be88c88a53
5,ESJEY5AKYJOXW2QBR4ZFS66VDJLR4X6H,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1898-04-25T12:00:00Z,[ger],f61e98b4-097a-46bc-9103-2ac0ad56c5c1
6,S4Z2EV2CDCXNYVHGJTYGGSNFSORBALSB,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1898-03-25T12:00:00Z,[ger],6c605931-e9e6-4403-ac97-0a98cf12562a
7,S7QRCY7RSSICNY3S5YVHTLDGXGEVOA5H,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1898-12-30T12:00:00Z,[ger],54c6de9b-5211-42cf-bbe8-9b039d71edae
8,JK6XH2EIPE53DMJJT5TB6O4YMEMF6RKE,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1897-03-25T12:00:00Z,[ger],b1d691cb-6f08-405e-8cce-4518d1c2efef
9,QAB2J4UV3OLBYP6PCENI455G36FWZ7TC,Die Radlerin : internationales Sportblatt der ...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2782137-7,False,1898-04-10T12:00:00Z,[ger],f2d03402-1177-4507-ab13-0ec23323913f


### Erste Datenanalyse

In dem Dataframe können nun Datenanalysen vorgenommen werden. Wir wollen den Publikationszeitraum von der Zeitung „Die Radlerin“ ermitteln.

In [30]:
# Sicherstellen, dass publication_date als Datumswerte formatiert sind
df['publication_date'] = pd.to_datetime(df['publication_date'])

# Frühestes und spätestes Datum ermitteln
earliest_date = df['publication_date'].min()
latest_date = df['publication_date'].max()

# Ergebnisse anzeigen
print(f"Frühestes Veröffentlichungsdatum: {earliest_date}")
print(f"Spätestes Veröffentlichungsdatum: {latest_date}")

Frühestes Veröffentlichungsdatum: 1897-02-25 12:00:00+00:00
Spätestes Veröffentlichungsdatum: 1898-12-30 12:00:00+00:00


## Download der METS/MODS-Daten

Im Deutschen Zeitungsportal ist jede Zeitungsausgabe durch eine METS/MODS-Datei repräsentiert. Diese beinhaltet Links zu den Bild- und Volltextseiten sowie weitere Informationen.

> METS ([Metadata Encoding and Transmission Standard](https://www.loc.gov/standards/mets/)) und MODS ([Metadata Object Description Schema](https://www.loc.gov/standards/mods/)) sind zwei miteinander verbundene Metadatenformate, die häufig in Bibliotheken und Archiven verwendet werden.
>
> METS ist ein XML-basiertes Format, das zur Kodierung und Übertragung von Metadaten für digitale Bibliotheksobjekte entwickelt wurde. Es dient als Container, der verschiedene Arten von Metadaten und die strukturellen Beziehungen zwischen den Teilen eines digitalen Objekts beschreibt. Es besteht aus mehreren Abschnitten wie `metsHdr` (Header), `fileSec` (Dateien), `structMap` (Strukturkarte) und `metadataSec` (Metadatensektion).
> 
> MODS ist ebenfalls ein XML-basiertes Format, das zur Erfassung und zum Austausch bibliografischer Informationen entwickelt wurde. Es bietet Elemente wie `titleInfo` (Titelinformation), `name` (Namen), `originInfo` (Herkunftsinformationen), und `subject` (Themen).

Auf die METS/MODS-Daten kann über die [API-Methode](https://labs.deutsche-digitale-bibliothek.de/app/ddbapi/#/items/getItemsIdSourceComponent) `items/{id}/source/record` zugegriffen werden. Dafür muss die DDB-ID der Zeitungsausgabe bekannt sein, die wir im vorhergehenden Schritt (bspw. `HG7ELL5U3S2JP3NB5YN7HDH6TLUN2FJ7`) ermittelt haben. Damit ergibt sich folgende API-Abfrage:

- https://api.deutsche-digitale-bibliothek.de/2/items/HG7ELL5U3S2JP3NB5YN7HDH6TLUN2FJ7/source/record

### KI-generierter Programmcode
<div class="alert alert-block alert-info">
<b>Prompt:</b> Erstelle mit Python ein Verzeichnis, in dem heruntergeladene XML-Dateien gespeichert werden können. Iteriere durch jede Zeile des bestehenden DataFrames <code>df</code>, der die Spalten <code>id</code> und <code>publication_date</code> enthält.
Für jede Zeile:
<ol>
  <li>Extrahiere den Wert der Spalte <code>id</code>.</li>
  <li>Formatiere den DateTime-Wert der Spalte <code>publication_date</code> im Format <code>YYYY-MM-DD</code>.</li>
  <li>Generiere eine URL <code>https://api.deutsche-digitale-bibliothek.de/2/items/{id}/source/record</code> zur API-Abfrage</li>
  <li>Setze die HTTP-Header so, dass die Antwort im XML-Format akzeptiert wird.</li>
  <li>Erstelle einen Dateipfad für die XML-Datei im Format <code>{publication_date}_{id}.xml</code> und speichere sie im erstellten Verzeichnis.</li>
</ol>
</div>

In [31]:
# Verzeichnis für die XML-Dateien erstellen
directory = 'xml_files'
if not os.path.exists(directory):
    os.makedirs(directory)

# Durch jede Zeile des DataFrames iterieren
for index, row in df.iterrows():
    # Extrahiere den Wert der Spalte id
    item_id = row['id']
    
    # Formatiere den Wert der Spalte publication_date im Format YYYY-MM-DD
    publication_date = row['publication_date'].strftime('%Y-%m-%d')
    
    # Generiere die URL zur API-Abfrage
    url = f'https://api.deutsche-digitale-bibliothek.de/2/items/{item_id}/source/record'
    
    # Setze die HTTP-Header
    headers = {
        'Accept': 'application/xml'
    }
    
    # API-Anfrage senden
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Erstelle einen Dateipfad für die XML-Datei
        file_path = os.path.join(directory, f'{publication_date}_{item_id}.xml')
        
        # Speichere die XML-Datei im erstellten Verzeichnis
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Datei gespeichert: {file_path}')
    else:
        print(f'Fehler beim Abrufen der Datei für ID {item_id}: {response.status_code}')

print('Fertig!')

Datei gespeichert: xml_files/1898-12-30_HG7ELL5U3S2JP3NB5YN7HDH6TLUN2FJ7.xml
Datei gespeichert: xml_files/1897-02-25_6N7GKKGH5IPAYYTBNHCG6RM4PZ6F6TWF.xml
Datei gespeichert: xml_files/1897-03-25_F47A6H5QGYWDZE5JVHTUPCVI5TJWDBFX.xml
Datei gespeichert: xml_files/1897-07-10_KUVZNUNQH5S4P44J76HYAKBQUAFV5LSI.xml
Datei gespeichert: xml_files/1898-04-10_TPV722VIMVEUEHQJJARAX7PFZOVNYRMQ.xml
Datei gespeichert: xml_files/1898-04-25_ESJEY5AKYJOXW2QBR4ZFS66VDJLR4X6H.xml
Datei gespeichert: xml_files/1898-03-25_S4Z2EV2CDCXNYVHGJTYGGSNFSORBALSB.xml
Datei gespeichert: xml_files/1898-12-30_S7QRCY7RSSICNY3S5YVHTLDGXGEVOA5H.xml
Datei gespeichert: xml_files/1897-03-25_JK6XH2EIPE53DMJJT5TB6O4YMEMF6RKE.xml
Datei gespeichert: xml_files/1898-04-10_QAB2J4UV3OLBYP6PCENI455G36FWZ7TC.xml
Datei gespeichert: xml_files/1898-11-15_66W66KJM4WUQOMISTE4QPFBKEDE22YRQ.xml
Datei gespeichert: xml_files/1898-04-25_KLJ2VSSBU4LH2WXGNAB6BFBZKIVXADJJ.xml
Datei gespeichert: xml_files/1898-02-25_Y2DFATJRMRWKFG7WKWRIHHPLVDGSPN2D.xml

## Download der Bild- und Volltextdaten

Die heruntergeladenen METS/MODS-Daten enthalten Verlinkungen zu allen Bild- und Volltextdaten. Eine stark verkürzte und schematische Darstellung der METS/MODS-Struktur, die insbesondere die Positionen von `mets:fileSec` und `mets:fileGrp` zeigt, sieht wie folgt aus. Hier muss man beachten, dass eine `mets:fileGrp` immer eine zusammengehörige Gruppe von Dateien bildet. Es sind `DEFAULT` (Link zu Bilddateien beim Datenpartner der Deutschen Digitalen Bibliothek) und `DDB_FULLTEXT` (Link zu den Volltextdaten bei der Deutschen Digitalen Bibliothek) interessant.

```XML
<mets:mets xmlns:mets="http://www.loc.gov/METS/" xmlns:mods="http://www.loc.gov/mods/v3">
  
  <!-- Header -->
  <mets:metsHdr CREATEDATE="2024-07-30T12:00:00">
    <!-- Agent Information -->
  </mets:metsHdr>

  <!-- Descriptive Metadata -->
  <mets:dmdSec ID="dmd001">
    <mets:mdWrap MDTYPE="MODS">
      <mets:xmlData>
        <mods:mods>
          <!-- MODS Metadata -->
        </mods:mods>
      </mets:xmlData>
    </mets:mdWrap>
  </mets:dmdSec>

  <!-- File Section -->
  <mets:fileSec>
    <mets:fileGrp USE="DEFAULT">
      <mets:file ID="file0001" MIMETYPE="image/jpeg">
        <mets:FLocat LOCTYPE="URL" xlink:href="http://example.com/image1.jpg"/>
      </mets:file>
    </mets:fileGrp>
    <mets:fileGrp USE="DDB_FULLTEXT">
      <mets:file ID="file0002" MIMETYPE="text/xml">
        <mets:FLocat LOCTYPE="URL" xlink:href="http://example.com/fulltext1.xml"/>
      </mets:file>
    </mets:fileGrp>
  </mets:fileSec>

  <!-- Structural Map -->
  <mets:structMap TYPE="logical">
    <mets:div TYPE="document" DMDID="dmd001">
      <mets:div TYPE="page" ORDER="1">
        <mets:fptr FILEID="file0001"/>
      </mets:div>
      <mets:div TYPE="page" ORDER="2">
        <mets:fptr FILEID="file0002"/>
      </mets:div>
    </mets:div>
  </mets:structMap>

</mets:mets>

```

#### xPath-Querys

Eine Möglichkeit diese Informationen mit einer Abfragesprache für XML-Daten abzufragen ist [xPath](https://de.wikipedia.org/wiki/XPath). Die Abfragen (Querys) sehen wie folgt aus:

- DEFAULT-Bilder: `//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href`
- DDB_FULLTEXT-Volltexte: `//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href`

### KI-generierter Programmcode

<div class="alert alert-block alert-info">
<p><b>Prompt:</b> Erstelle ein möglichst einfaches Python-Skript, das alle XML-Dateien in dem Verzeichnis <code>xml_files</code> einliest und URLs mittels des XPath-Ausdrucks <code>//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href</code> extrahiert. Die extrahierten URLs sollen heruntergeladen und in Unterverzeichnissen gespeichert werden. Die Unterverzeichnisse werden nach den XML-Dateien benannt. Die heruntergeladenen XML-Dateien sollen mit 1 beginnend durchnummeriert werden.</p>

<p>Das Gleiche soll mit JPEG-Dateien und dem xPath-Ausdruck <code>//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href</code> gemacht werden.</p>
</div>

In [32]:
import os
import requests
from lxml import etree

# Definiere die Namensräume
NAMESPACES = {
    'mets': 'http://www.loc.gov/METS/',
    'xlink': 'http://www.w3.org/1999/xlink'
}

# Verzeichnisse definieren
xml_directory = 'xml_files'
download_directory = 'downloads'

# Erstelle das Download-Verzeichnis, falls es nicht existiert
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Funktion, um URLs aus einer XML-Datei zu extrahieren und herunterzuladen
def download_files_from_xml(xml_file_path, xpath_expr, subfolder, extension):
    # Lade die XML-Datei ein
    with open(xml_file_path, 'rb') as xml_file:
        tree = etree.parse(xml_file)

    # Extrahiere die URLs mit dem gegebenen XPath-Ausdruck
    urls = tree.xpath(xpath_expr, namespaces=NAMESPACES)

    # Erstelle das Unterverzeichnis, benannt nach der XML-Datei
    xml_file_name = os.path.splitext(os.path.basename(xml_file_path))[0]
    destination_dir = os.path.join(download_directory, xml_file_name, subfolder)
    os.makedirs(destination_dir, exist_ok=True)

    # Lade jede URL herunter und speichere die Datei
    for i, url in enumerate(urls, start=1):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Überprüfe auf HTTP-Fehler
            file_path = os.path.join(destination_dir, f'{i}.{extension}')
            with open(file_path, 'wb') as output_file:
                output_file.write(response.content)
            print(f'Downloaded {url} to {file_path}')
        except requests.RequestException as e:
            print(f'Failed to download {url}: {e}')

# Durchlaufe alle XML-Dateien im Verzeichnis
for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_file_path = os.path.join(xml_directory, xml_file)
        # Lade und speichere Dateien mit dem ersten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href', 'DEFAULT', 'jpeg')
        # Lade und speichere Dateien mit dem zweiten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href', 'DDB_FULLTEXT', 'xml')


Downloaded https://digital.slub-dresden.de/data/kitodo/DieRa_41188932X-18970225/DieRa_41188932X-18970225_tif/jpegs/00000001.tif.medium.jpg to downloads/1897-02-25_6N7GKKGH5IPAYYTBNHCG6RM4PZ6F6TWF/DEFAULT/1.jpeg
Downloaded https://digital.slub-dresden.de/data/kitodo/DieRa_41188932X-18970225/DieRa_41188932X-18970225_tif/jpegs/00000002.tif.medium.jpg to downloads/1897-02-25_6N7GKKGH5IPAYYTBNHCG6RM4PZ6F6TWF/DEFAULT/2.jpeg
Downloaded https://digital.slub-dresden.de/data/kitodo/DieRa_41188932X-18970225/DieRa_41188932X-18970225_tif/jpegs/00000003.tif.medium.jpg to downloads/1897-02-25_6N7GKKGH5IPAYYTBNHCG6RM4PZ6F6TWF/DEFAULT/3.jpeg
Downloaded https://digital.slub-dresden.de/data/kitodo/DieRa_41188932X-18970225/DieRa_41188932X-18970225_tif/jpegs/00000004.tif.medium.jpg to downloads/1897-02-25_6N7GKKGH5IPAYYTBNHCG6RM4PZ6F6TWF/DEFAULT/4.jpeg
Downloaded https://digital.slub-dresden.de/data/kitodo/DieRa_41188932X-18970225/DieRa_41188932X-18970225_tif/jpegs/00000005.tif.medium.jpg to downloads/1897